In [1]:
# import xgboost as xgb
import numpy as np
from tqdm import tqdm
from itertools import *
from scipy.stats import uniform, randint
import json
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

In [2]:
with open("./data_sno_test_set.txt", "r") as f:
    predict_stns = f.read().split()
    predict_stns.sort()
print("predict_stns:", predict_stns)
stn_pre = 0

predict_stns: ['500101001', '500101002', '500101003', '500101004', '500101005', '500101006', '500101007', '500101008', '500101009', '500101010', '500101013', '500101014', '500101015', '500101018', '500101019', '500101020', '500101021', '500101022', '500101023', '500101024', '500101025', '500101026', '500101027', '500101028', '500101029', '500101030', '500101031', '500101032', '500101033', '500101034', '500101035', '500101036', '500101037', '500101038', '500101039', '500101040', '500101041', '500101042', '500101091', '500101092', '500101093', '500101094', '500101114', '500101115', '500101123', '500101166', '500101175', '500101176', '500101181', '500101184', '500101185', '500101188', '500101189', '500101190', '500101191', '500101193', '500101199', '500101209', '500101216', '500101219', '500105066', '500106002', '500106003', '500106004', '500119043', '500119044', '500119045', '500119046', '500119047', '500119048', '500119049', '500119050', '500119051', '500119052', '500119053', '500119054

In [3]:
def Err_func(b_predict, b_truth, total):
    return 3 * abs(b_predict - b_truth) / total * ( abs((3 * b_truth - total)/(3 *total)) + abs((3 * b_truth - 2 * total)/(3 * total)) )

In [4]:
import tensorflow as tf
def val(predic_y, y, total):
  err = 0
  for i in range(len(y)):
    err += Err_func(float(predic_y[i]), float(y[i]), float(total))

  return err / len(y)

In [5]:
def loss(yp, yt):
    return Err_func(float(yp), float(yt), float(stn_pre))

In [6]:
def Load_stn_tot():
  with open("./data_stn_tot.json", 'r') as f:
    stn_tot = json.load(f)
  return dict(stn_tot)

In [7]:
def read_data(stn, mode):
  data = []
  x = []
  y = []
  path = f"./data/{stn}_{mode}.txt"

  with open(path, 'r') as f:
    for line in f.readlines():
      tmp_list = line.split()
      for i in range(len(tmp_list)):
        if i not in [0, 1, 2]:
          tmp_list[i] = float(tmp_list[i])
      # if tmp_list[1] in week_list:
      min = int(tmp_list[2][:2]) * 60 + int(tmp_list[2][3:])
      weekday = int(tmp_list[4])
      all_min = 1440 * (weekday - 1) + min
      if tmp_list[4] == 5 and min > 1440 - 60 * 6:
        tmp_list[5] = 0

      def sin_a(a, b):
        return np.sin(a * 2 * np.pi / b)
      def cos_a(a, b):
        return np.cos(a * 2 * np.pi / b)
      k = 1440
      x_input = []
      for i in range(1, 10):
        x_input.append(sin_a(all_min, k / i))
        x_input.append(cos_a(all_min, k / i))
      x_input = [sin_a(min, k), cos_a(min, k), sin_a(all_min, k * 7), cos_a(all_min, k * 7)] + tmp_list[5:8]
      
      x.append(list(x_input))
      y.append(int(tmp_list[10]))
      
      frac = int(tmp_list[10])/ tmp_list[3]
      N = 6
      if mode == 'train':
        for i in range(int(N * 3 * ( abs(frac - 1/3) + abs(frac - 2/3))) - N):
          x.append(list(x_input))
          y.append(int(tmp_list[10]))
          # if min < 60 * 7 or min > 1440 - 60 * 2:
          #   x.append(list(x_input))
          #   y.append(int(tmp_list[10]))
        # if min < 60 * 7 or min > 1440 - 60 * 2:
        #     x.append(list(x_input))
        #     y.append(int(tmp_list[10]))
          
      data.append(tmp_list)
      
  # print("data:", data)
  # print("x_train:", x)
  # print("y_train:", y)
  data = np.array(data)
  x = np.array(x)
  y = np.array(y)
  scaler = StandardScaler()
  x = scaler.fit_transform(x)
  return data, x, y

In [8]:
import tensorflow as tf
from keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits

all_val_err = 0
all_train_err = 0
count = 0
predict_file = open("./MLP_predict.csv", 'w')
predict_file.write("id,sbi\n")
stn_tot = Load_stn_tot()
# predict_stns = ['500101008']
val_MLP_json = {}
for stn in predict_stns:
  global stn_pre 
  stn_pre= stn_tot[stn]
  count += 1
  data_train, x_train, y_train = read_data(stn, 'train')
  data_val, x_val, y_val = read_data(stn, 'val')
  data_test, x_test, y_test = read_data(stn, 'test')

  # Train the model
  
  scaler = StandardScaler()
  model = models.Sequential()
  model.add(layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)))
  model.add(layers.Dense(64, activation='relu'))
  model.add(layers.Dense(32, activation='relu'))
  model.add(layers.Dense(1))  # Assuming 10 classes for this example

  # Compile the model
  model.compile(optimizer='adam', 
                loss='mean_squared_error' 
                # loss=loss
                ) # For classification tasks


  # Train the model 
  model.fit(x_train, y_train, epochs=3)
  # model.fit(x_train, y_train)
  pre_train = model.predict(x_train)

  
  pre_val = model.predict(x_val)
  train_err = val(pre_train, y_train, int(stn_tot[stn]))
  val_err = val(pre_val, y_val, int(stn_tot[stn]))
  
  all_train_err += train_err
  all_val_err += val_err
  print(f"{stn}: train: {train_err:<19}, \033[34mval: {val_err:<19}\033[0m, avg_train: {all_train_err / count:<19}, avg_val: {all_val_err/count:<19}")
  val_MLP_json[stn] = {}
  val_MLP_json[stn]['val'] = val_err
  val_MLP_json[stn]['train'] = train_err
  
  # plt.figure(figsize=(80, 5), dpi=300)
  # plt.plot(pre_train, label='pre_train', color='b')
  # plt.plot(y_train, label='truth', color='darkorange')
  # plt.legend()
  # plt.show()
  
  # plt.figure(figsize=(80, 5), dpi=300)
  # plt.plot(pre_val, label='pre_train', color='b')
  # plt.plot(y_val, label='truth', color='darkorange')
  # plt.legend()
  # plt.show()
  
  pre_test = model.predict(x_test)
  for i in range(len(data_test)):
    if data_test[i][2][3:] in ["00", "20", "40"]:
      id = f'{data_test[i][1]}_{int(data_test[i][0])}_{data_test[i][2]}'
      predict_file.write(f"{id},{int(pre_test[i])}\n")

predict_file.close()
print(f"final: train: {all_train_err / len(predict_stns)}, val: {all_val_err / len(predict_stns)}")
with open("./val_MLP.json", 'w') as f:
  json.dump(val_MLP_json, f, indent=2)


Epoch 1/3
12106/12106 [==============================] - 9s 740us/step - loss: 19.4292
Epoch 2/3
12106/12106 [==============================] - 8s 641us/step - loss: 16.7549
Epoch 3/3
315/315 [==============================] - 0s 793us/step


/var/folders/59/0dhfqywn3jld8bd1k8dfjg640000gn/T/ipykernel_29659/2686421191.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  err += Err_func(float(predic_y[i]), float(y[i]), float(total))


500101001: train: 0.19571043542801078, val: 0.2777079547718238 , avg_train: 0.19571043542801078, avg_val: 0.2777079547718238 
495/495 [==============================] - 0s 315us/step


/var/folders/59/0dhfqywn3jld8bd1k8dfjg640000gn/T/ipykernel_29659/2769502317.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predict_file.write(f"{id},{int(pre_test[i])}\n")


Epoch 1/3
13112/13112 [==============================] - 9s 665us/step - loss: 8.2122
Epoch 2/3
13112/13112 [==============================] - 9s 696us/step - loss: 7.2116
Epoch 3/3
315/315 [==============================] - 0s 611us/step


...

/var/folders/59/0dhfqywn3jld8bd1k8dfjg640000gn/T/ipykernel_29659/2769502317.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predict_file.write(f"{id},{int(pre_test[i])}\n")


Epoch 1/3
15140/15140 [==============================] - 8s 503us/step - loss: 11.0075
Epoch 2/3
15140/15140 [==============================] - 8s 504us/step - loss: 10.6718
Epoch 3/3
315/315 [==============================] - 0s 295us/step


/var/folders/59/0dhfqywn3jld8bd1k8dfjg640000gn/T/ipykernel_29659/2686421191.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  err += Err_func(float(predic_y[i]), float(y[i]), float(total))


500119091: train: 0.2513921486377548 , val: 0.27824083264262217, avg_train: 0.22435674657978483, avg_val: 0.30003616151484014
495/495 [==============================] - 0s 391us/step
final: train: 0.22435674657978483, val: 0.30003616151484014


/var/folders/59/0dhfqywn3jld8bd1k8dfjg640000gn/T/ipykernel_29659/2769502317.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predict_file.write(f"{id},{int(pre_test[i])}\n")
